We can begin analyzing molecules by looking at the molecules in QM7. The QM7 dataset is a subset of GDB-13 (a database of nearly 1 billion stable and synthetically accessible organic molecules) composed of all molecules of up to 23 atoms (including 7 heavy atoms C, N, O, and S), totalling 7165 molecules. It contains the Coulomb matrix representation of these molecules and their atomization energies computed similarly to the FHI-AIMS implementation of the Perdew-Burke-Ernzerhof hybrid functional (PBE0). This dataset features a large variety of molecular structures such as double and triple bonds, cycles, carboxy, cyanide, amide, alcohol and epoxy.

To download and load the QM7 dataset as a Python dictionary, run:

In [ ]:
!wget http://quantum-machine.org/data/qm7.mat
!mv qm7.mat ../data/.

raw_qm7 = scipy.io.loadmat("../data/qm7.mat") # the qm7 dataset will be loaded as a Python dictionary

The dataset is composed of five multidimensional arrays:
- X (7165 x 23 x 23) - Coulomb matrices
- T (7165) - atomization energies (the labels)
- P (5 x 1433) - cross-validation splits
- Z (7165) - atomic charge of each atom in the molecules
- R (7165 x 3) - Cartesian coordinates of each atom in the molecules

In [ ]:
X = raw_qm7["X"] # Coulomb matrices
T = raw_qm7["T"] # atomization energies
P = raw_qm7["P"] # cross-validation splits
Z = raw_qm7["Z"] # atomic numbers
R = raw_qm7["R"] # coordinates

To make it a bit easier to work with the molecules, let's convert them into RDKit ´Mol´ objects, and respectively into the corresponding SMILES:

In [ ]:
# # Get the molobjs
# atoms       = Z
# coordinates = R
#
# mols   = []
# smiles = []
# for idx, atom in enumerate(atoms):
#     mol = xyz2mol.xyz2mol(atom, coordinates[idx], allow_charged_fragments=False)
#     mols.append(mol)
#     smiles.append(Chem.MolToSmiles(mol))

smiles = open("../4-Autoencoders/smiles.txt").read().split()
mols   = [Chem.MolFromSmiles(smi) for smi in smiles]

Let's look at some of the SMILES from QM7:

In [ ]:
from IPython.display import display

print("methane:", smiles[0])
methane_img       = Draw.MolToImage(mols[0])
display(methane_img)

print("2-aminoacetaldehyde:", smiles[100])
aminoacetaldehyde_img = Draw.MolToImage(mols[100])
display(aminoacetaldehyde_img)

print("1-methylcyclopropan-1-ol:", smiles[200])
methylcyclopropanol_img       = Draw.MolToImage(mols[200])
display(methylcyclopropanol_img)



## Coulomb Matrices
Another way to represent molecular structures in cheminformatics is Coulomb matrices (CMs), where each element of the matrix captures the Coulombic interaction between pairs of atoms in a molecule, combining nuclear charge and interatomic distances. Below we show an example CM for water.

In [ ]:
import numpy as np

# atomic numbers
Z_O = 8  # Oxygen
Z_H = 1  # Hydrogen

# distance between Oxygen and Hydrogen in Angstroms (approximate)
R_OH = 0.96

# Coulomb matrix for water (H2O)
# initialize the matrix
coulomb_matrix = np.zeros((3, 3))

# diagonal elements: 0.5 * Z^2.4
coulomb_matrix[0, 0] = 0.5 * Z_O ** 2.4  # Oxygen
coulomb_matrix[1, 1] = 0.5 * Z_H ** 2.4  # Hydrogen 1
coulomb_matrix[2, 2] = 0.5 * Z_H ** 2.4  # Hydrogen 2

# off-diagonal elements: Z_i * Z_j / R_ij
# assuming the distance between each hydrogen and the oxygen is the same
coulomb_matrix[0, 1] = coulomb_matrix[1, 0] = Z_O * Z_H / R_OH  # oxygen-hydrogen 1
coulomb_matrix[0, 2] = coulomb_matrix[2, 0] = Z_O * Z_H / R_OH  # oxygen-hydrogen 2
coulomb_matrix[1, 2] = coulomb_matrix[2, 1] = Z_H * Z_H / R_OH  # hydrogen-hydrogen

coulomb_matrix